In [39]:
!pip3 install elasticsearch
!pip3 install pymongo

In [40]:
from pymongo import MongoClient

In [41]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import pandas as pd
import json


import numpy as np

LOCAL = False

es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch"])

#df_billboard = pd.read_json("./newscrawler/newscrawler/spiders/scrap200.json")

In [42]:
client = MongoClient()
db = client["client_name"]
billboard = db["billboard"]

In [43]:
df_billboard = pd.DataFrame(list(billboard.find()))
df_billboard.head()

,_id,album,artist,rank
0,5e4d127160024d16b100c1a7,Please Excuse Me For Being Antisocial,Roddy Ricch,1
1,5e4d127160024d16b100c1a8,Hollywood's Bleeding,Post Malone,2
2,5e4d127160024d16b100c1a9,Music To Be Murdered By,Eminem,3
3,5e4d127160024d16b100c1aa,Father Of All...,Green Day,4
4,5e4d127160024d16b100c1ab,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,5


In [44]:
df_billboard_clean = df_billboard.drop(labels='_id',axis='columns')
documents = df_billboard_clean.fillna("").to_dict(orient="records")

In [45]:
documents[0:1]

[{'album': 'Please Excuse Me For Being Antisocial',
  'artist': 'Roddy Ricch',
  'rank': '1'}]

In [46]:
type(toString(documents[0:1]))

NameError: name 'toString' is not defined

In [47]:
def generate_data(documents):
    for docu in documents:
        yield {
            "_index": "albums",
            "_type": "album",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es_client, generate_data(documents))

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7faba6557470>: Failed to establish a new connection: [Errno -2] Name or service not known) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7faba6557470>: Failed to establish a new connection: [Errno -2] Name or service not known)

In [48]:
#!curl http://localhost:9200/_cat/indices?v

In [49]:
#!curl http://localhost:9200/albums/_search

In [50]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "Malone",
      "fields": [ "artist" ] 
    }
  }
}

In [51]:
result = es_client.search(index="albums", body=QUERY)
[elt['_source']['album'] for elt in result["hits"]["hits"]]

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7faba655a828>: Failed to establish a new connection: [Errno -2] Name or service not known) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7faba655a828>: Failed to establish a new connection: [Errno -2] Name or service not known)

In [52]:
!python3 music_search.py

 * Serving Flask app "music_search" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:2745/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 107-460-003
127.0.0.1 - - [19/Feb/2020 18:10:32] "GET / HTTP/1.1" 302 -
music_search.py:38: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
  form = SearchBar()
127.0.0.1 - - [19/Feb/2020 18:10:32] "GET /MusicSearch HTTP/1.1" 200 -
music_search.py:38: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
  form = SearchBar()
127.0.0.1 - - [19/Feb/2020 18:10:34] "POST /MusicSearch HTTP/1.1" 302 -
127.0.0.1 - - [19/Feb/2020 18:10:34] "GET /information/Taylor%20Swift HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/flask/app.py", line 2309, in __call__
    return self.wsgi_app(environ, 